In [50]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
from scipy.cluster.hierarchy import linkage, fcluster
from sklearn.preprocessing import StandardScaler

df = pd.read_csv('MjS-data-lente2025.csv', sep='\t')
df2 = df[['id', 'timestamp', 'longitude', 'latitude', 'temperature']]
df2['longitude'] = df2.groupby('id')['longitude'].transform(lambda x: x.mode().iloc[0] if not x.mode().empty else x.iloc[0])
df2['latitude'] = df2.groupby('id')['latitude'].transform(lambda x: x.mode().iloc[0] if not x.mode().empty else x.iloc[0])
df2['timestamp'] = pd.to_datetime(df2['timestamp'])
ids_with_nan = df2.loc[df2['longitude'].isna() | df2['latitude'].isna(), 'id'].unique()
gdf = df2[~df2['id'].isin(ids_with_nan)].copy()
print(gdf)

         id           timestamp  longitude  latitude  temperature
0       258 2025-03-01 00:00:11    5.13159   52.0839       3.3750
1       782 2025-03-01 00:02:40    5.12436   52.1092       3.6875
2       792 2025-03-01 00:02:54    5.12326   52.0702       2.9375
3       745 2025-03-01 00:04:55    6.78931   52.2802      15.4375
4       791 2025-03-01 00:04:56    5.11829   52.0925       4.4375
...     ...                 ...        ...       ...          ...
156513  840 2025-05-31 23:54:16    5.07104   52.0903      19.5625
156514  418 2025-05-31 23:54:58    5.07642   52.1198      20.3125
156515  254 2025-05-31 23:58:35    5.04443   52.0804      18.9375
156516  719 2025-05-31 23:59:28    5.12106   52.1052      20.1875
156517  724 2025-05-31 23:59:59    5.00449   52.1041      18.6875

[156022 rows x 5 columns]


C:\Users\bcywl\AppData\Local\Temp\ipykernel_27524\3577659370.py:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\bcywl\AppData\Local\Temp\ipykernel_27524\3577659370.py:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\bcywl\AppData\Local\Temp\ipykernel_27524\3577659370.py:14: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pa

In [51]:
import dash
from dash import dcc, html, Output, Input
import geopandas as gpd
import pandas as pd
import plotly.express as px

# App setup
app = dash.Dash(__name__)
app.title = "Dataclub Dashboard"

app.layout = html.Div([
    html.H1("Interactief Dashboard"),
    
    dcc.Dropdown(
        id='group-select',
        options=[
            {'label': 'Per kastje', 'value': 'sensor'},
            {'label': 'Per wijk', 'value': 'region'}
        ],
        value='sensor'
    ),

    dcc.DatePickerRange(
        id='date-picker',
        start_date=gdf['timestamp'].min(),
        end_date=gdf['timestamp'].max()
    ),

    html.Div(style={'display': 'flex'}, children=[

        dcc.Graph(
            id='map',
            style={'height': '600px', 'width': '800px'}
        ),

        dcc.Graph(
            id='temp-plot',
            style={'height': '600px', 'width': '600px'}
        ),

    ])
])


@app.callback(
    Output('map', 'figure'),
    [Input('group-select', 'value'),
     Input('date-picker', 'start_date'),
     Input('date-picker', 'end_date')]
)
def update_map(group_type, start_date, end_date):
    filtered = gdf[(gdf['timestamp'] >= start_date) & (gdf['timestamp'] <= end_date)]

    if group_type == 'region':
        hover_name = 'region'
        # Show all points with real temp, but for region you can assign region as hover_name
        geo = filtered.copy()
    else:
        hover_name = 'id'
        geo = filtered.copy()

    map_fig = px.scatter_mapbox(
        geo,
        lat='latitude',
        lon='longitude',
        hover_name=hover_name,
        zoom=11,
        mapbox_style="open-street-map"
    )
    return map_fig


@app.callback(
    Output('temp-plot', 'figure'),
    [Input('map', 'clickData'),
     Input('group-select', 'value'),
     Input('date-picker', 'start_date'),
     Input('date-picker', 'end_date')]
)
def update_temp_plot(click_data, group_type, start_date, end_date):
    filtered = gdf[(gdf['timestamp'] >= start_date) & (gdf['timestamp'] <= end_date)]

    if not click_data:
        return px.line(title="Klik op een punt op de kaart om de temperatuur te bekijken.")

    selected_value = click_data['points'][0]['hovertext']

    if group_type == 'region':
        df_filtered = filtered[filtered['region'] == selected_value]
        color_col = 'region'
    else:
        df_filtered = filtered[filtered['id'] == selected_value]
        color_col = 'id'

    temp_fig = px.line(
        df_filtered,
        x='timestamp',
        y='temperature',
        color=color_col,
        title=f"Temperatuur over tijd - {selected_value}"
    )

    return temp_fig


if __name__ == '__main__':
    app.run(debug=True)